In [1]:
import xarray as xr 

# ds = xr.open_dataset('/teamspace/studios/this_studio/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/f0e93607df96295f1c9e14ccfe96032f.nc')
ds = xr.open_dataset('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/ERA5_monthly_averaged_data_on_single_levels.nc')

In [2]:
ds

<xarray.Dataset> Size: 44GB
Dimensions:    (date: 405, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) int64 3kB 19910101 19910201 19910301 ... 20240801 20240901
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (date) <U4 6kB ...
Data variables: (12/26)
    u10        (date, latitude, longitude) float32 2GB ...
    v10        (date, latitude, longitude) float32 2GB ...
    t2m        (date, latitude, longitude) float32 2GB ...
    sst        (date, latitude, longitude) float32 2GB ...
    sp         (date, latitude, longitude) float32 2GB ...
    tp         (date, latitude, longitude) float32 2GB ...
    ...         ...
    lai_lv     (date, latitude, longitude) float32 2GB ...
    cvl        (date, latitude, longitude) float32 2GB ...
    tvh        (date, latitude, longitude) float32 2GB ...
    tvl        (date, latitude, longitude) float32 2GB ...
    z          (date, latitude, longitude) float32 2GB ...
    lsm        (date, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-25T11:10 GRIB to CDM+CF via cfgrib-0.9.1...

# Format datatype

In [3]:
import pandas as pd

# 1. Convert 'date' coordinate to datetime
ds['date'] = pd.to_datetime(ds['date'].astype(str), format='%Y%m%d')

# Fill na values

In [4]:
# INTO 3.data_preprocessing.ipynb 

import xarray as xr
from typing import List, Union

def fillna_in_variables(
    ds: xr.Dataset,
    variables: List[str],
    fill_value: Union[int, float]
) -> xr.Dataset:
    """
    Fills NaN values in specified variables of an xarray.Dataset with a provided value.

    Args:
        ds (xr.Dataset): The input dataset containing the data variables.
        variables (List[str]): A list of variable names in the dataset for which to fill NaN values.
        fill_value (Union[int, float]): The value to fill NaN values with (e.g., 0).

    Returns:
        xr.Dataset: A new dataset with NaN values filled in the specified variables.
    
    Raises:
        ValueError: If a variable in `variables` does not exist in the dataset.
    
    Example:
        filled_ds = fillna_in_variables(ds, ["t2m", "swvl1"], fill_value=0)
    """
    # Check if each variable exists in the dataset
    for var in variables:
        if var not in ds:
            raise ValueError(f"Variable '{var}' not found in the dataset.")
    
    # Fill NaN values for the specified variables
    filled_ds = ds.copy()
    for var in variables:
        filled_ds[var] = filled_ds[var].fillna(fill_value)
    
    return filled_ds

ds = fillna_in_variables(ds, ["sst"], fill_value=0)

# Drop irrelavat variables 

In [5]:
# INTO 3.data_preprocessing.ipynb 
ds = ds.drop_vars(['swvl1', 'swvl2', 'swvl4', 'lai_lv', 'cvl', 'tvl', 'stl1'])

# Interploation

In [6]:
# import numpy as np
# import xarray as xr

# # Step 1: Define target grid with 1° resolution
# target_lat = np.arange(-90, 90.1, 1)   # From -90 to 90 degrees inclusive
# target_lon = np.arange(0, 360, 1)      # From 0 to 359.5 degrees inclusive

# # Step 2: Create target grid Dataset (optional, for reference)
# target_grid = xr.Dataset(
#     {
#         "latitude": (["latitude"], target_lat),
#         "longitude": (["longitude"], target_lon),
#     }
# )

# # Step 3: Ensure latitude is ascending
# if ds.latitude[0] > ds.latitude[-1]:
#     ds = ds.sortby("latitude")

# # Step 4: Perform interpolation
# ds = ds.interp(latitude=target_lat, longitude=target_lon, method="linear")


In [9]:
ds

<xarray.Dataset> Size: 32GB
Dimensions:    (date: 405, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 3kB 1991-01-01 1991-02-01 ... 2024-09-01
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (date) <U4 6kB ...
Data variables: (12/19)
    u10        (date, latitude, longitude) float32 2GB ...
    v10        (date, latitude, longitude) float32 2GB ...
    t2m        (date, latitude, longitude) float32 2GB ...
    sst        (date, latitude, longitude) float32 2GB 271.5 271.5 ... 0.0 0.0
    sp         (date, latitude, longitude) float32 2GB ...
    tp         (date, latitude, longitude) float32 2GB ...
    ...         ...
    swvl3      (date, latitude, longitude) float32 2GB ...
    cvh        (date, latitude, longitude) float32 2GB ...
    lai_hv     (date, latitude, longitude) float32 2GB ...
    tvh        (date, latitude, longitude) float32 2GB ...
    z          (date, latitude, longitude) float32 2GB ...
    lsm        (date, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-25T11:10 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
# ds.to_netcdf('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/02_intermediate/merged_data.nc')
ds.to_netcdf('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/02_intermediate/preprocessed_data.nc')